In [28]:
import pandas as pd
import numpy as np
from scipy import stats

In [7]:
df1 = pd.read_csv('Data for MDE - opened normal.csv')
df1

,date,user_id
0,01.01.2023,101232
1,01.01.2023,101233
2,01.01.2023,101234
3,01.01.2023,101235
4,01.01.2023,101236
...,...,...
2946,05.02.2023,104178
2947,05.02.2023,104179
2948,05.02.2023,104180
2949,05.02.2023,104181


In [34]:
df_count = df1.groupby('date').agg({'user_id': 'count'})
df_count.user_id.mean()


81.97222222222223

In [3]:
df2 = pd.read_csv('Data for MDE - clicked normal.csv')
df2

,date,user_id
0,01.01.2023,101240
1,01.01.2023,101241
2,01.01.2023,101242
3,01.01.2023,101243
4,01.01.2023,101244
...,...,...
275,05.02.2023,104123
276,05.02.2023,104124
277,05.02.2023,104125
278,05.02.2023,104126


In [5]:
df3 = pd.read_csv('Data for MDE - opened sale.csv')
df3

,date,user_id
0,06.02.2023,104183
1,06.02.2023,104184
2,06.02.2023,104185
3,06.02.2023,104186
4,06.02.2023,104187
...,...,...
582,12.02.2023,104765
583,12.02.2023,104766
584,12.02.2023,104767
585,12.02.2023,104768


In [6]:
df4 = pd.read_csv('Data for MDE - clicked sale.csv')
df4

,date,user_id
0,06.02.2023,104220
1,06.02.2023,104221
2,06.02.2023,104222
3,06.02.2023,104223
4,06.02.2023,104224
...,...,...
80,12.02.2023,104718
81,12.02.2023,104719
82,12.02.2023,104720
83,12.02.2023,104721


In [31]:
import statsmodels.stats.api as sms
from math import ceil

effect_size = sms.proportion_effectsize(0.095, 0.145)    # Расчет ожидаемого эффекта на основе исторических данных (имели 10%, хотим 12%)

required_n = sms.NormalIndPower().solve_power(
    effect_size, 
    power=0.8, 
    alpha=0.05, 
    ratio=1
    )                                                  # Рассчитываем необходимый размер каждой группы

required_n = ceil(required_n)                          # Округляем до следующего целого значения                         

print(required_n)

657


In [24]:
ab = pd.read_csv('Data for results1.csv')
ab

,date,group,user_id,converted
0,01.03.2023,control,106085,1
1,01.03.2023,control,106086,1
2,01.03.2023,control,106087,1
3,01.03.2023,control,106088,1
4,01.03.2023,control,106089,1
...,...,...,...,...
1200,05.03.2023,treatment,107285,0
1201,05.03.2023,treatment,107286,0
1202,05.03.2023,treatment,107287,0
1203,05.03.2023,treatment,107288,0


In [25]:
ab['group'].value_counts()

control      613
treatment    592
Name: group, dtype: int64

In [29]:
conversion_rates = ab.groupby('group')['converted']

std_p = lambda x: np.std(x, ddof=0)              # Стандартное отклонение
se_p = lambda x: stats.sem(x, ddof=0)            # Стандартная ошибка

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

,conversion_rate,std_deviation,std_error
group,,,
control,0.100,0.299,0.012
treatment,0.137,0.344,0.014


In [30]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = ab[ab['group'] == 'control']['converted']
treatment_results = ab[ab['group'] == 'treatment']['converted']
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: -2.01
p-value: 0.045
ci 95% for control group: [0.076, 0.123]
ci 95% for treatment group: [0.109, 0.165]
